In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"parkjeongyeol","key":"2829620be97f119eda3004d8b67e9f7c"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         75           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         79           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        595           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        136           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1480           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      14681            True  
house-pr

In [3]:
!kaggle datasets download -d kneroma/tacotrashdataset

100% 2.78G/2.79G [00:16<00:00, 174MB/s]
100% 2.79G/2.79G [00:16<00:00, 182MB/s]


In [4]:
!unzip /content/tacotrashdataset.zip

Archive:  /content/tacotrashdataset.zip
  inflating: best-checkpoint-003epoch.bin  
  inflating: data/annotations.json   
  inflating: data/batch_1/000000.jpg  
  inflating: data/batch_1/000001.jpg  
  inflating: data/batch_1/000003.jpg  
  inflating: data/batch_1/000004.jpg  
  inflating: data/batch_1/000005.jpg  
  inflating: data/batch_1/000006.jpg  
  inflating: data/batch_1/000007.jpg  
  inflating: data/batch_1/000008.jpg  
  inflating: data/batch_1/000010.jpg  
  inflating: data/batch_1/000011.jpg  
  inflating: data/batch_1/000012.jpg  
  inflating: data/batch_1/000013.jpg  
  inflating: data/batch_1/000014.jpg  
  inflating: data/batch_1/000015.jpg  
  inflating: data/batch_1/000016.jpg  
  inflating: data/batch_1/000017.jpg  
  inflating: data/batch_1/000019.jpg  
  inflating: data/batch_1/000021.jpg  
  inflating: data/batch_1/000022.jpg  
  inflating: data/batch_1/000023.jpg  
  inflating: data/batch_1/000024.jpg  
  inflating: data/batch_1/000025.jpg  
  inflating: data/ba

In [5]:
!pip install pycocotools

## Faster RCNN

In [8]:
!git clone https://github.com/wimlds-trojmiasto/detect-waste.git

Cloning into 'detect-waste'...
remote: Enumerating objects: 928, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 928 (delta 71), reused 80 (delta 36), pack-reused 766
Receiving objects: 100% (928/928), 85.18 MiB | 18.67 MiB/s, done.
Resolving deltas: 100% (437/437), done.


In [9]:
import numpy as np
import os
os.chdir('/content/detect-waste/FastRCNN')
import torch
import pycocotools
import torch.utils.data
from PIL import Image
import pandas as pd
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
from PIL import ImageDraw
import os.path as path
import re
import json

In [10]:
def parse_one_annot(path_to_data_file, filename):
    data = pd.read_csv(path_to_data_file)
    print(filename)
    boxes_array = data[data["filename"] == filename][["xmin", "ymin",        
    "xmax", "ymax"]].values
    print(boxes_array)
    return boxes_array

In [11]:
def getListOfFiles(dirName):
    listOfFile = os.listdir(dirName)
    allFiles = list()
    for entry in listOfFile:
        fullPath = os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
                fullPath = fullPath.replace(data_folder, '')
                allFiles.append(fullPath)  
    return allFiles

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, root, data_file, transforms=None):
        self.root = root
        self.transforms = transforms
        files = getListOfFiles(os.path.join(root))
        p = re.compile('batch')
        l2 = [ s for s in files if p.match(s) ]
        self.imgs = sorted(l2)
        self.path_to_data_file = data_file
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.imgs[idx])
        self.imgs[idx] = self.imgs[idx]
        img = Image.open(img_path).convert("RGB")
        box_list = parse_one_annot(self.path_to_data_file, self.imgs[idx])
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
                 img, target = self.transforms(img, target)
        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [13]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
    return model

In [14]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

## CSV 파일 만들기

In [15]:
data_file = '/content/data/annot.csv'
json_path = '/content/data/annotations.json'
annot_folder = '/content/data'
with open(json_path) as json_file: 
    data = json.load(json_file)

In [16]:
def export_data(data, dtype, file):
    img = data[dtype] 
    data_file = open(file, 'w') 
    csv_writer = csv.writer(data_file) 
    count = 0
    for i in img: 
        if count == 0: 
            header = i.keys() 
            csv_writer.writerow(header) 
            count += 1
        csv_writer.writerow(i.values()) 
    data_file.close()

In [17]:
import csv
export_data(data, 'images', os.path.join(annot_folder,'data_file.csv'))
export_data(data, 'annotations', os.path.join(annot_folder,'data_file2.csv'))
export_data(data, 'categories', os.path.join(annot_folder,'data_file3.csv'))

In [18]:
data3 = pd.read_csv(os.path.join(annot_folder,'data_file3.csv'))
data2 = pd.read_csv(os.path.join(annot_folder,'data_file2.csv'))
data = pd.read_csv(os.path.join(annot_folder,'data_file.csv'))

In [19]:
data = data.rename(columns={"id": "image_id"})
data3 = data3.rename(columns={"id": "category_id"})
data2 = data2.merge(data, on='image_id', how='left')
data2 = data2.merge(data3, on='category_id', how='left')
data2 = data2.rename(columns={"supercategory": "class"})
data2 = data2.rename(columns={"file_name": "filename"})
data2['bbox'] = data2['bbox'].map(lambda x: x.lstrip('[').rstrip(']'))
data2[['xmin','ymin','xmax','ymax']] = data2.bbox.str.split(",",expand=True,)

In [20]:
data2['xmax'] = data2['xmax'].astype(float) + data2['width'].astype(float)
data2['ymax'] = data2['ymax'].astype(float) + data2['height'].astype(float)

In [21]:
data2 = data2[['filename','width','height','class','xmin','ymin','xmax','ymax']]
data2.to_csv(data_file,index=False)

In [22]:
data_folder = '/content/data/'
data_file = '/content/data/annot.csv'
working_dir = '/content/detect-waste/FastRCNN'
num_classes = 28

dataset = Dataset(root= data_folder, data_file= data_file , transforms = get_transform(train=True))
dataset_test = Dataset(root= data_folder, data_file= data_file , transforms = get_transform(train=False))

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
testing_num = int(len(dataset)*0.2)
dataset = torch.utils.data.Subset(dataset, indices[:-testing_num])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-testing_num:])
data_loader = torch.utils.data.DataLoader(
              dataset, batch_size=2, shuffle=True, num_workers=4,
              collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(
         dataset_test, batch_size=1, shuffle=False, num_workers=4,
         collate_fn=utils.collate_fn)
print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

We have: 1500 examples, 1200 are training and 300 testing


In [23]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model(num_classes)
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                       step_size=3,gamma=0.1)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


batch_15/000020.jpg
[[ 980.  734. 2310. 1869.]
 [1091.  967. 2100. 1616.]
 [  39.  433. 2251. 1627.]]
batch_2/000026.JPG
[[ 424. 1378. 3408. 2577.]
 [ 720.  915. 3435. 2587.]
 [1973.  957. 3398. 2576.]
 [3194.  964. 3333. 2542.]
 [3147.  999. 3348. 2534.]
 [2617.  907. 3542. 2628.]]batch_6/000032.JPG

[[ 951. 1516. 3501. 2604.]
 [2202. 1638. 3520. 2549.]
 [ 360. 1917. 3369. 2509.]
 [1316. 1943. 3373. 2546.]]
batch_14/000090.jpg
[[1073. 1935. 3163. 4357.]]
batch_1/000091.JPG
[[   0.  414. 5454. 4049.]
 [2593.  914. 3846. 2889.]]
batch_8/000061.jpg
[[ 562.  801. 1297. 1794.]
 [ 917.    0. 1226. 1617.]
 [ 678.  718. 1225. 1666.]]
batch_13/000074.jpg
[[ 915.  318. 5909. 5290.]]
batch_15/000084.jpg
[[1059.  699. 3608. 3642.]
 [1632.  301. 3691. 3720.]
 [1570. 1215. 3842. 2874.]]
batch_8/000034.jpg
[[1747.  538. 2603. 3408.]
 [2321. 2300. 2543. 3305.]
 [1503.  944. 2488. 3288.]
 [  50. 1684. 2509. 3304.]
 [ 430. 1855. 2472. 3312.]
 [ 968. 2200. 2513. 3309.]
 [1116. 2041. 2524. 3338.]
 [1335.

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Streaming output truncated to the last 5000 lines.
 [1084. 3396. 3043. 5355.]
 [1474. 3787. 3047. 5375.]
 [2158. 1906. 3022. 5339.]
 [ 662. 2820. 3028. 5353.]
 [ 118. 2243. 3005. 5350.]]
batch_12/000025.jpg
[[ 990.  862. 3485. 5653.]
 [1182. 2443. 3457. 5773.]
 [ 199. 1959. 3203. 5516.]
 [2865. 1648. 3051. 5337.]
 [2687. 2094. 3047. 5399.]
 [2698. 2372. 3059. 5387.]
 [2860. 2865. 3113. 5352.]
 [2910. 3129. 3061. 5385.]
 [2721. 3037. 3070. 5377.]
 [2638. 3709. 3153. 5399.]
 [2541. 4956. 3029. 5366.]
 [1587. 4683. 3104. 5433.]
 [1405. 4091. 3000. 5363.]
 [1494. 4503. 3017. 5359.]
 [ 927. 4686. 3065. 5390.]
 [1007. 4706. 3025. 5396.]
 [ 140. 1180. 3056. 5405.]
 [ 103. 1242. 3050. 5425.]
 [2166. 2560. 3103. 5468.]
 [2251. 3197. 3047. 5437.]
 [2359. 3077. 3069. 5423.]
 [1828. 3101. 3067. 5437.]
 [1868. 3305. 3057. 5381.]
 [1122. 2956. 3031. 5401.]
 [1183. 2363. 3100. 5407.]
 [1571.  850. 3104. 5368.]
 [ 330. 2330. 3035. 5342.]]
batch_12/000026.jpg
[[ 999.  636. 3068. 5381.]
 [ 952.  851. 30

RuntimeError: ignored

In [ ]:
model_path = '/content/drive/MyDrive/인공지능 그랜드 챌린지/model.h5'

torch.save(model.state_dict(), model_path)

In [ ]:
loaded_model = get_model(num_classes = num_classes)
loaded_model.load_state_dict(torch.load(model_path))

In [ ]:
idx = 1
img, _ = dataset_test[idx]
label_boxes = np.array(dataset_test[idx][1]["boxes"])
loaded_model.eval()
with torch.no_grad():
    prediction = loaded_model([img])
image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
draw = ImageDraw.Draw(image)
for elem in range(len(label_boxes)):
    draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
    (label_boxes[elem][2], label_boxes[elem][3])], 
    outline ="green", width =3)
for element in range(len(prediction[0]["boxes"])):
    boxes = prediction[0]["boxes"][element].cpu().numpy()
    score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                    decimals= 4)
    if score > 0.8:
        draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
        outline ="red", width =3)
        draw.text((boxes[0], boxes[1]), text = str(score))
image